<a href="https://colab.research.google.com/github/RifatMuhtasim/Data_Science/blob/main/Feature_Engineering/1.Create_New_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()

df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
df['target'] = iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
# When two columns are numerical

correlation_matrix = df.corr()
correlation_with_target = correlation_matrix.loc['sepal length (cm)', 'target']
correlation_with_target

0.7825612318100821

In [4]:
# When 2 Columns are Categorical

from scipy.stats import chi2_contingency

def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    r, k = confusion_matrix.shape
    phi2 = chi2 / n
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Create a contingency table
conf_matrix = pd.crosstab(df['sepal length (cm)'], df['target'])

# Calculate Cramér's V
cramers_v_value = cramers_v(conf_matrix.values)

print("Cramér's V value:", cramers_v_value)

Cramér's V value: 0.5446864194565882


In [5]:
# When target is binary and features are continuous

from scipy.stats import pointbiserialr

correlation, p_value = pointbiserialr(df['sepal length (cm)'], df['target'])
correlation, p_value

(0.7825612318100813, 2.890478352614215e-32)

In [6]:
# When target variable is categorical and features are continuous

def correlation_ratio(categories, measurements):
    # Convert categorical labels to integers
    categories = np.array(categories)
    unique_categories = np.unique(categories)
    category_counts = {category: np.sum(categories == category) for category in unique_categories}

    # Total sum of squares
    total_variance = np.var(measurements) * (len(measurements) - 1)

    # Between-group sum of squares
    numerator = 0
    for category, count in category_counts.items():
        category_measurements = measurements[categories == category]
        numerator += count * np.var(category_measurements)

    # Calculate correlation ratio
    eta = numerator / total_variance
    return eta


# correlation_eta = correlation_ratio(target, variable)
# print("Correlation Ratio (Eta):", correlation_eta)

# Polynomial Features
---

In [7]:
def Polynomial_Features(df, target):
    result_df = df.copy()
    column_list = [col for col in df.columns]
    column_list.remove(target)

    for i in column_list:
        result_df[i + '_square'] = df[i] ** 2
        result_df[i + '_cube'] = df[i] ** 3
        result_df[i + '_sqrt'] = np.sqrt(df[i])
        result_df[i + '_log'] = np.log(df[i])
    return result_df

In [8]:
pdf = Polynomial_Features(df, 'target')

In [9]:
pdf.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,sepal length (cm)_square,sepal length (cm)_cube,sepal length (cm)_sqrt,sepal length (cm)_log,sepal width (cm)_square,...,sepal width (cm)_sqrt,sepal width (cm)_log,petal length (cm)_square,petal length (cm)_cube,petal length (cm)_sqrt,petal length (cm)_log,petal width (cm)_square,petal width (cm)_cube,petal width (cm)_sqrt,petal width (cm)_log
0,5.1,3.5,1.4,0.2,0,26.01,132.651,2.258318,1.629241,12.25,...,1.870829,1.252763,1.96,2.744,1.183216,0.336472,0.04,0.008,0.447214,-1.609438
1,4.9,3.0,1.4,0.2,0,24.01,117.649,2.213594,1.589235,9.00,...,1.732051,1.098612,1.96,2.744,1.183216,0.336472,0.04,0.008,0.447214,-1.609438
2,4.7,3.2,1.3,0.2,0,22.09,103.823,2.167948,1.547563,10.24,...,1.788854,1.163151,1.69,2.197,1.140175,0.262364,0.04,0.008,0.447214,-1.609438
3,4.6,3.1,1.5,0.2,0,21.16,97.336,2.144761,1.526056,9.61,...,1.760682,1.131402,2.25,3.375,1.224745,0.405465,0.04,0.008,0.447214,-1.609438
4,5.0,3.6,1.4,0.2,0,25.00,125.000,2.236068,1.609438,12.96,...,1.897367,1.280934,1.96,2.744,1.183216,0.336472,0.04,0.008,0.447214,-1.609438


# Create Feature using Multiple Methods (Interaction)

In [10]:
pd.set_option("display.max_columns", None)

In [11]:
def Interaction_Features_for_multiple_columns(df, target):
    result_df = df.copy()
    column_list = [col for col in df.columns]
    column_list.remove(target)

    for i in column_list:
        for col in column_list:
            if i not in col:
                result_df[i + '+' + col] = df[i] + df[col]
                result_df[i + '-' + col] = df[i] - df[col]
                result_df[i + '*' + col] = df[i] * df[col]
                result_df[i + '/' + col] = df[i] / df[col]
    return result_df

In [12]:
mdf = Interaction_Features_for_multiple_columns(pdf, 'target')

In [13]:
mdf.head()

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  sepal length (cm)_square  sepal length (cm)_cube  \
0       0                     26.01                 132.651   
1       0                     24.01                 117.649   
2       0                     22.09                 103.823   
3       0                     21.16                  97.336   
4       0                     25.00                 125.000   

   sepal length (cm)_sqrt  sepal length (cm)_log  sepal width (cm)_square  \
0                2.258318               1.629241                    12.25   
1                2.213594               1.589235                     9.00   
2                2.167948               1.547563                    10.24   
3                2.144761               1.526056                     9.61   
4                2.236068               1.609438                    12.96   

   sepal width (cm)_cube  sepal width (cm)_sqrt  sepal width (cm)_log  \
0                 42.875               1.870829              1.252763   
1                 27.000               1.732051              1.098612   
2                 32.768               1.788854              1.163151   
3                 29.791               1.760682              1.131402   
4                 46.656               1.897367              1.280934   

   petal length (cm)_square  petal length (cm)_cube  petal length (cm)_sqrt  \
0                      1.96                   2.744                1.183216   
1                      1.96                   2.744                1.183216   
2                      1.69                   2.197                1.140175   
3                      2.25                   3.375                1.224745   
4                      1.96                   2.744                1.183216   

   petal length (cm)_log  petal width (cm)_square  petal width (cm)_cube  \
0               0.336472                     0.04                  0.008   
1               0.336472                     0.04                  0.008   
2               0.262364                     0.04                  0.008   
3               0.405465                     0.04                  0.008   
4               0.336472                     0.04                  0.008   

   petal width (cm)_sqrt  petal width (cm)_log  \
0               0.447214             -1.609438   
1               0.447214             -1.609438   
2               0.447214             -1.609438   
3               0.447214             -1.609438   
4               0.447214             -1.609438   

   sepal length (cm)+sepal width (cm)  sepal length (cm)-sepal width (cm)  \
0                                 8.6                                 1.6   
1                                 7.9                                 1.9   
2                                 7.9                                 1.5   
3                                 7.7                                 1.5   
4                                 8.6                                 1.4   

   sepal length (cm)*sepal width (cm)  sepal length (cm)/sepal width (cm)  \
0                               17.85                            1.457143   
1                               14.70                            1.633333   
2                               15.04                            1.468750   
3                               14.26                            1.483871   
4                               18.00                            1.388889   

   sepal length (cm)+petal length (cm)  sepal length (cm)-petal length (cm)  \
0        

# Binning or Discretization

In [14]:
from sklearn.preprocessing import KBinsDiscretizer

# Define number of bins
n_bins = 6
kbins = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
binned_data = kbins.fit_transform(np.array(mdf['sepal length (cm)']).reshape(-1, 1))
mdf['sepal area bins'] = binned_data
mdf['sepal area bins'] = mdf['sepal area bins'].astype(int)

In [15]:
# Define number of bins
n_bins = 6
kbins = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
binned_data = kbins.fit_transform(np.array(mdf['petal length (cm)']).reshape(-1, 1))
mdf['petal area bins'] = binned_data
mdf['petal area bins'] = mdf['petal area bins'].astype(int)

In [16]:
mdf.head()

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  sepal length (cm)_square  sepal length (cm)_cube  \
0       0                     26.01                 132.651   
1       0                     24.01                 117.649   
2       0                     22.09                 103.823   
3       0                     21.16                  97.336   
4       0                     25.00                 125.000   

   sepal length (cm)_sqrt  sepal length (cm)_log  sepal width (cm)_square  \
0                2.258318               1.629241                    12.25   
1                2.213594               1.589235                     9.00   
2                2.167948               1.547563                    10.24   
3                2.144761               1.526056                     9.61   
4                2.236068               1.609438                    12.96   

   sepal width (cm)_cube  sepal width (cm)_sqrt  sepal width (cm)_log  \
0                 42.875               1.870829              1.252763   
1                 27.000               1.732051              1.098612   
2                 32.768               1.788854              1.163151   
3                 29.791               1.760682              1.131402   
4                 46.656               1.897367              1.280934   

   petal length (cm)_square  petal length (cm)_cube  petal length (cm)_sqrt  \
0                      1.96                   2.744                1.183216   
1                      1.96                   2.744                1.183216   
2                      1.69                   2.197                1.140175   
3                      2.25                   3.375                1.224745   
4                      1.96                   2.744                1.183216   

   petal length (cm)_log  petal width (cm)_square  petal width (cm)_cube  \
0               0.336472                     0.04                  0.008   
1               0.336472                     0.04                  0.008   
2               0.262364                     0.04                  0.008   
3               0.405465                     0.04                  0.008   
4               0.336472                     0.04                  0.008   

   petal width (cm)_sqrt  petal width (cm)_log  \
0               0.447214             -1.609438   
1               0.447214             -1.609438   
2               0.447214             -1.609438   
3               0.447214             -1.609438   
4               0.447214             -1.609438   

   sepal length (cm)+sepal width (cm)  sepal length (cm)-sepal width (cm)  \
0                                 8.6                                 1.6   
1                                 7.9                                 1.9   
2                                 7.9                                 1.5   
3                                 7.7                                 1.5   
4                                 8.6                                 1.4   

   sepal length (cm)*sepal width (cm)  sepal length (cm)/sepal width (cm)  \
0                               17.85                            1.457143   
1                               14.70                            1.633333   
2                               15.04                            1.468750   
3                               14.26                            1.483871   
4                               18.00                            1.388889   

   sepal length (cm)+petal length (cm)  sepal length (cm)-petal length (cm)  \
0        

# Correlation

1. Point-Biserial Correlation: This method calculates the correlation between a binary variable (representing the class labels in a classification problem) and a continuous variable. It is essentially a special case of Pearson correlation, where one of the variables is dichotomous.
2. Cramer's V: Cramer's V is a measure of association for categorical variables. It ranges from 0 to 1, with higher values indicating stronger association between variables. It is commonly used for assessing the association between categorical features and the target variable in classification problems.
3. Correlation Ratio (eta or Eta-squared): The correlation ratio measures the strength and direction of association between a categorical variable and a continuous variable. It is used when the target variable is categorical and the predictor variable is continuous.
4. Feature Importance: Instead of correlation coefficients, you can use feature importance techniques specific to classification algorithms. For example, decision trees and ensemble methods (e.g., Random Forest, Gradient Boosting) inherently provide feature importance scores based on how much each feature contributes to improving predictive performance.
5. Mutual Information: Mutual information measures the amount of information obtained about one variable through another variable. It can be used to assess the dependency between features and the target variable, irrespective of the variable types.

In [17]:
def Classification_correlation_ratio(df, target):
    def correlation_ratio(categories, measurements):
        # Convert categorical labels to integers
        categories = np.array(categories)
        unique_categories = np.unique(categories)
        category_counts = {category: np.sum(categories == category) for category in unique_categories}
        # Total sum of squares
        total_variance = np.var(measurements) * (len(measurements) - 1)

        # Between-group sum of squares
        numerator = 0
        for category, count in category_counts.items():
            category_measurements = measurements[categories == category]
            numerator += count * np.var(category_measurements)

        # Calculate correlation ratio
        eta = numerator / total_variance
        return eta

    columns = [col for col in df.columns]
    correlation_df = pd.DataFrame(columns=['column', 'value'])
    for i in columns:
        correlation_eta = correlation_ratio(df[target], df[i])
        correlation_df = pd.concat([correlation_df, pd.DataFrame({'column': [i], 'value': [correlation_eta]})], ignore_index=True)

    correlation_df = correlation_df[correlation_df['column'] != target]
    return correlation_df

In [18]:
correlation_df = Classification_correlation_ratio(mdf, 'target')

In [19]:
correlation_df.sort_values("value", ascending=False)

,column,value
1228,petal width (cm)_square/petal length (cm)_cube,0.996513
386,sepal length (cm)_cube-petal length (cm)_cube,0.977308
966,petal length (cm)_cube-sepal length (cm)_cube,0.977308
30,sepal length (cm)-petal width (cm),0.975930
202,petal width (cm)-sepal length (cm),0.975930
...,...,...
1312,petal width (cm)_cube/petal length (cm)_log,NaN
1324,petal width (cm)_cube/petal width (cm)_log,NaN
1388,petal width (cm)_sqrt/petal length (cm)_log,NaN
1400,petal width (cm)_sqrt/petal width (cm)_log,NaN


In [20]:
removed_columns = correlation_df[(correlation_df['value'] > 0.9) | (correlation_df['value'] < 0.1) | (correlation_df['value'].isna())]['column'].tolist()
mdf.drop(removed_columns, axis="columns", inplace=True)

In [21]:
mdf.head()

sepal length (cm)  sepal width (cm)  target  sepal length (cm)_square  \
0                5.1               3.5       0                     26.01   
1                4.9               3.0       0                     24.01   
2                4.7               3.2       0                     22.09   
3                4.6               3.1       0                     21.16   
4                5.0               3.6       0                     25.00   

   sepal length (cm)_cube  sepal length (cm)_sqrt  sepal length (cm)_log  \
0                 132.651                2.258318               1.629241   
1                 117.649                2.213594               1.589235   
2                 103.823                2.167948               1.547563   
3                  97.336                2.144761               1.526056   
4                 125.000                2.236068               1.609438   

   sepal width (cm)_square  sepal width (cm)_cube  sepal width (cm)_sqrt  \
0                    12.25                 42.875               1.870829   
1                     9.00                 27.000               1.732051   
2                    10.24                 32.768               1.788854   
3                     9.61                 29.791               1.760682   
4                    12.96                 46.656               1.897367   

   sepal width (cm)_log  petal length (cm)_square  petal length (cm)_cube  \
0              1.252763                      1.96                   2.744   
1              1.098612                      1.96                   2.744   
2              1.163151                      1.69                   2.197   
3              1.131402                      2.25                   3.375   
4              1.280934                      1.96                   2.744   

   petal width (cm)_square  petal width (cm)_cube  \
0                     0.04                  0.008   
1                     0.04                  0.008   
2                     0.04                  0.008   
3                     0.04                  0.008   
4                     0.04                  0.008   

   sepal length (cm)+sepal width (cm)  sepal length (cm)-sepal width (cm)  \
0                                 8.6                                 1.6   
1                                 7.9                                 1.9   
2                                 7.9                                 1.5   
3                                 7.7                                 1.5   
4                                 8.6                                 1.4   

   sepal length (cm)*sepal width (cm)  sepal length (cm)/sepal width (cm)  \
0                               17.85                            1.457143   
1                               14.70                            1.633333   
2                               15.04                            1.468750   
3                               14.26                            1.483871   
4                               18.00                            1.388889   

   sepal length (cm)+petal length (cm)  sepal length (cm)*petal length (cm)  \
0                                  6.5                                 7.14   
1                                  6.3                                 6.86   
2                                  6.0                                 6.11   
3                                  6.1                                 6.90   
4                                  6.4                                 7.00   

   sepal length (cm)+petal width (cm)  sepal length (cm)*petal width (cm)  \
0                                 5.3                                1.02   
1                                 5.1                                0.98   
2                                 4.9                                0.94   
3                                 4.8                                0.92   
4                                 5.2                           